In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.webdriver import FirefoxBinary
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from pyquery import PyQuery as pq
import re
import csv
import time

In [2]:
#(1).保存csv文件
def WriteDictToCSV(csv_file,csv_columns,dict_data):
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in dict_data:
            writer.writerow(data)
    return    

In [3]:
#(2).第一级爬虫(浏览器(假的),你要保存的数据内容)
def get_data(browser,list_to_save):
	html = browser.page_source#html 是网页的源代码
	doc = pq(html)#转换成PyQuery格式
	items = doc('#s_position_list .item_con_list .con_list_item').items()#找到职位的block 15个放到list
	for item in items:#遍历15个
		url = item.find('.p_top .position_link').attr('href')
		all_title = item.find('.p_top').text()
        
		lines = all_title.split('\n')
		title = lines[0]
		address = lines[2]
		time = lines[4]
# 		print(lines[0],'\n',lines[2],'\n',lines[4])
        
		all_tag = item.find('.li_b_l').text().replace('/ ', '').split(' ')
		salary = all_tag[0]        
		experience = all_tag[1]
		        
		education = all_tag[2]
		tags = [tag for tag in all_tag[3:]]
		company = item.find('.company_name').text()
		fuli = item.find('.li_b_r').text()
          
		data = {
			'title': title,
			'url': url,
			'company': company,
			'address': address,
			'salary': salary,
			'experience': experience,
			'education': education,
			'tags': tags,
			#'type': type_name,
			'fuli': fuli,
		}
		list_to_save.append(data)

In [4]:
#(3).第二级爬虫(传入个浏览器) 已经进去了
def craw_details(browser):
    for session in browser.find_elements_by_class_name('job_detail'):
        d = browser.find_element_by_class_name('job_bt')
        return d.text

In [11]:
#(4).得到url,准备进去.(一级爬好的链接)
def get_detais(url_list):
        detail_to_add = ''
        driver = webdriver.Chrome()
        driver.set_page_load_timeout(100)
        temp = []
        interation = 0 
        for block in url_list:
            block1 = block
            print('准备解析地址: ',block['url'],'\n')
            driver.get(block['url'])
            #执行爬虫
            detail_to_add = craw_details(driver)
            
            block1['detail'] = detail_to_add
            temp.append(block1) ####
            #新的表格 输出应该ok
            interation = interation +1
            print(interation)
        return temp

In [12]:
#class 是 类 (函数与变量的集合)
class LagouSpider:
    list_to_save = []
    #1.直接跳过来,初始化
    def __init__(self, city='全国', search_key='slam'):
        self.city = city
        self.search_key = search_key
#2.
    def get_jobs(self, driver):
        try:
            # 选择城市,在刚弹出来的戳深圳
            select_area = WebDriverWait(driver, 10).until(
                EC.visibility_of(driver.find_element_by_partial_link_text(self.city))
            )
       
            select_area.click()
            print('选择城市:', self.city)
        except Exception as e:
            driver.close()
            print(e)

        try:
            search = driver.find_element(By.ID, 'search_input')
            # 搜索岗位,输入职位名称
            search.send_keys(self.search_key)
            search.send_keys(Keys.RETURN)
            print('搜索岗位:', self.search_key)
            time.sleep(3)
        except Exception as e:
            driver.close()
            print(e)

        page = 1
        
        while True:
            try:
                # 判断是否含有职位item
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'con_list_item')))
            except Exception as e:
                print('没有找到职位item,自动中断')
                print(e)
                driver.close()
                break
             #得到网页源代码   
            doc = pq(driver.page_source)
            print('开始解析第 {} 页数据'.format(page))
            i = 0 
            # 主要在这里进行解析!!!!!!!!!!
            
            #下面代码调到(2)
            get_data(driver,LagouSpider.list_to_save)
            #print(LagouSpider.list_to_save)
            
            
#             for session in driver.find_elements_by_class_name('con_list_item'):
#                 article = {}
#                 job_name = session.find_element_by_tag_name('h3')
#                 money = session.find_element_by_class_name('money')           
#                print(money.text)             

            try:
                # 判断下一页按钮是否可以点击
                WebDriverWait(driver, 10).until(
                    EC.invisibility_of_element_located((By.CLASS_NAME, 'pager_next_disabled')))
                next_page = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'pager_next')))
                next_page.click()
                page += 1
                time.sleep(3)
            except Exception as e:
                print('没有下一页了,自动中断')
                #详细进行爬,内容爬虫***************
                new_detail = get_detais(LagouSpider.list_to_save)
                #!!!todo 改成新的表格
#                 header= ['title','url','company','address','salary','experience','education','tags','fuli']
#                 WriteDictToCSV('lagou.csv',header,LagouSpider.list_to_save)  
                ####保存一下数据
                header_new= ['title','url','company','address','salary','experience','education','tags','fuli','detail']
                WriteDictToCSV('lagou_detail.csv',header_new,new_detail) 
                ##witer one
                print(e)
                driver.close()
                break
#3.
    def crawl(self, url='https://www.lagou.com/'):

        driver =webdriver.Chrome()
        driver.set_page_load_timeout(30)
        driver.get(url)
        #self代表在自己class里面跳,到2,传Chrome进去
        self.get_jobs(driver)


if __name__ == '__main__':
    spider = LagouSpider(city='全国', search_key='slam')
    #跳到LagouSpider 的1
    spider.crawl()
    #跳到LagouSpider 的 3

选择城市: 全国
搜索岗位: slam
开始解析第 1 页数据
开始解析第 2 页数据
开始解析第 3 页数据
开始解析第 4 页数据
开始解析第 5 页数据
开始解析第 6 页数据
开始解析第 7 页数据
开始解析第 8 页数据
开始解析第 9 页数据
开始解析第 10 页数据
开始解析第 11 页数据
开始解析第 12 页数据
开始解析第 13 页数据
开始解析第 14 页数据
开始解析第 15 页数据
开始解析第 16 页数据
开始解析第 17 页数据
开始解析第 18 页数据
开始解析第 19 页数据
开始解析第 20 页数据
开始解析第 21 页数据
开始解析第 22 页数据
开始解析第 23 页数据
没有下一页了,自动中断
准备解析地址:  https://www.lagou.com/jobs/5306190.html 

1
准备解析地址:  https://www.lagou.com/jobs/5311013.html 

2
准备解析地址:  https://www.lagou.com/jobs/5312480.html 

3
准备解析地址:  https://www.lagou.com/jobs/5330542.html 

4
准备解析地址:  https://www.lagou.com/jobs/5328704.html 

5
准备解析地址:  https://www.lagou.com/jobs/5325564.html 

6
准备解析地址:  https://www.lagou.com/jobs/5066442.html 

7
准备解析地址:  https://www.lagou.com/jobs/2051049.html 

8
准备解析地址:  https://www.lagou.com/jobs/4591256.html 

9
准备解析地址:  https://www.lagou.com/jobs/5273446.html 

10
准备解析地址:  https://www.lagou.com/jobs/3933234.html 

11
准备解析地址:  https://www.lagou.com/jobs/5291602.html 

12
准备解析地址:  https://www.lagou.com/jobs/517

146
准备解析地址:  https://www.lagou.com/jobs/4120299.html 

147
准备解析地址:  https://www.lagou.com/jobs/5262974.html 

148
准备解析地址:  https://www.lagou.com/jobs/5049697.html 

149
准备解析地址:  https://www.lagou.com/jobs/4426853.html 

150
准备解析地址:  https://www.lagou.com/jobs/5080201.html 

151
准备解析地址:  https://www.lagou.com/jobs/4848710.html 

152
准备解析地址:  https://www.lagou.com/jobs/3979345.html 

153
准备解析地址:  https://www.lagou.com/jobs/4964591.html 

154
准备解析地址:  https://www.lagou.com/jobs/5140815.html 

155
准备解析地址:  https://www.lagou.com/jobs/5259478.html 

156
准备解析地址:  https://www.lagou.com/jobs/3365787.html 

157
准备解析地址:  https://www.lagou.com/jobs/4120306.html 

158
准备解析地址:  https://www.lagou.com/jobs/5269666.html 

159
准备解析地址:  https://www.lagou.com/jobs/5273966.html 

160
准备解析地址:  https://www.lagou.com/jobs/995200.html 

161
准备解析地址:  https://www.lagou.com/jobs/5255442.html 

162
准备解析地址:  https://www.lagou.com/jobs/4204838.html 

163
准备解析地址:  https://www.lagou.com/jobs/4255588.html 

164
准备解析地址:

295
准备解析地址:  https://www.lagou.com/jobs/5276223.html 

296
准备解析地址:  https://www.lagou.com/jobs/4952417.html 

297
准备解析地址:  https://www.lagou.com/jobs/4734518.html 

298
准备解析地址:  https://www.lagou.com/jobs/4655505.html 

299
准备解析地址:  https://www.lagou.com/jobs/4060569.html 

300
准备解析地址:  https://www.lagou.com/jobs/5112251.html 

301
准备解析地址:  https://www.lagou.com/jobs/3926707.html 

302
准备解析地址:  https://www.lagou.com/jobs/4502915.html 

303
准备解析地址:  https://www.lagou.com/jobs/5254285.html 

304
准备解析地址:  https://www.lagou.com/jobs/4849064.html 

305
准备解析地址:  https://www.lagou.com/jobs/4606387.html 

306
准备解析地址:  https://www.lagou.com/jobs/4369350.html 

307
准备解析地址:  https://www.lagou.com/jobs/4667471.html 

308
准备解析地址:  https://www.lagou.com/jobs/5241933.html 

309
准备解析地址:  https://www.lagou.com/jobs/4813502.html 

310
准备解析地址:  https://www.lagou.com/jobs/5088855.html 

311
准备解析地址:  https://www.lagou.com/jobs/4574891.html 

312
准备解析地址:  https://www.lagou.com/jobs/4026849.html 

313
准备解析地址